In [1]:
%matplotlib inline
%load_ext autoreload

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

import sncosmo

from copy import deepcopy
import gedankenLSST

from LSSTmetrics import PerSNMetric
from lsst.sims.photUtils import BandpassDict

import analyzelcFits as anf

In [2]:
# Run this once. It does not change

lsst_bp = BandpassDict.loadTotalBandpassesFromFiles()

# sncosmo Bandpasses required for fitting
throughputsdir = os.getenv('THROUGHPUTS_DIR')

from astropy.units import Unit
bandPassList = ['u', 'g', 'r', 'i', 'z', 'y']
banddir = os.path.join(os.getenv('THROUGHPUTS_DIR'), 'baseline')

for band in bandPassList:

    # setup sncosmo bandpasses
    bandfname = banddir + "/total_" + band + '.dat'


    # register the LSST bands to the SNCosmo registry
    # Not needed for LSST, but useful to compare independent codes
    # Usually the next two lines can be merged,
    # but there is an astropy bug currently which affects only OSX.
    numpyband = np.loadtxt(bandfname)
    #print band
    sncosmoband = sncosmo.Bandpass(wave=numpyband[:, 0],
                                   trans=numpyband[:, 1],
                                   wave_unit=Unit('nm'),
                                   name=band)
    sncosmo.registry.register(sncosmoband, force=True)

In [3]:
lsstCadence = deepcopy(gedankenLSST.LSSTReq)

In [4]:
# function that produces expected value of variances for many shifted lightcurves
# sncosmo_lc


def shift_loop(delta_t0):
    var = []

    for i in delta_t0:
        try:
            sn = PerSNMetric(summarydf=lsst_obs.summary,t0=i, raCol='ra', decCol='dec', lsst_bp=lsst_bp)
            var.append(1.0/sn.qualityMetric(Disp=1.0))
            #print 'variance!', var[i]
        except:
            print('I failed!')
    
    return np.mean(np.array(var))


In [14]:
# use LSST cadence
# choose number of lightcurves to shift
# for each shifted lightcurve, evaluate at sampling locations (no measurement errors for now)
# for each shifted lightcurve, calculate variance of parameters, then sum them to get expected value of variance
# profit



def cadence_loop(bumps,delta_t0):
    #delta_t0 = np.linspace(snLSST.SN.mintime(), snLSST.SN.maxtime(), 10)
    #print delta_t0
    #print bumps
    length = []
    mu_variance_per_bump = []
    for i in bumps:
        print i
        lsstCadence['bF'] = i
        
        lsst_obs = gedankenLSST.GSN_Obs(mjd_center=49570., 
                                    lsstrequirements=lsstCadence,
                                    ra=58., dec=-27.,
                                    timeWindow=[-130., 150.]) 
        length.append(len(lsst_obs.summary))
        mu_variance_per_bump.append(shift_loop(delta_t0))
    
    return mu_variance_per_bump, length


In [ ]:
t0 = 49570.
dt = 50.
delta_t0 = np.linspace(t0-dt,t0+dt,10)
#49540.0
#49645.0

bumps = np.arange(0.1, 10, 0.5)
mvpb, length = cadence_loop(bumps,delta_t0)

0.1
20

/Users/karaponder/.local/lib/python2.7/site-packages/LSSTmetrics/metrics.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['SNR'] = df['flux'] / df['fluxerr']
/Users/karaponder/.local/lib/python2.7/site-packages/LSSTmetrics/metrics.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['DetectionEfficiency'] = df.apply(self.func, axis=1)
/Users/karaponder/.local/lib/python2.7/site-packages/LSSTmetrics/metrics.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d


19
21
19
20
20
19
21
19
20
0.6

## TESTING BELOW

In [7]:
## THIS IS WHAT YOU NEED TO CHANGE
lsstCadence['bF'] = 1


# time window increased to 200 days to make sure LC stays within time frame
lsst_obs = gedankenLSST.GSN_Obs(mjd_center=49570., 
                                    lsstrequirements=lsstCadence,
                                    ra=58., dec=-27.,
                                    timeWindow=[-130., 150.]) 

In [13]:
len(lsst_obs.summary)
#snLSST = PerSNMetric(summarydf=lsst_obs.summary,t0=49570., raCol='ra', decCol='dec', lsst_bp=lsst_bp)

68

In [20]:
1.0/ snLSST.qualityMetric(Disp=1.0) #= 0.05**2/sigma**2

0.011661010512237181

In [50]:
var = []
delta_t0 = np.linspace(snLSST.SN.mintime(), snLSST.SN.maxtime(), 10)

for i in delta_t0:
    try:
        sn = PerSNMetric(summarydf=lsst_obs.summary,t0=i, raCol='ra', decCol='dec', lsst_bp=lsst_bp)
        #sn.lcplot()
        #sn.SNCosmoLC()
        #print sn.qualityMetric(Disp=1.0)
        var.append(1.0/sn.qualityMetric(Disp=1.0))
    
        #print var
    except:
        print('I failed!')
    
mean_var = np.mean(np.array(var))

19
19
20
20
19
21
19
18
22
17


In [12]:
#print len(var), var, mean_var

print snLSST.SN.mintime()
print snLSST.SN.maxtime()

49540.0
49645.0


In [13]:
# function that takes a set of parameters, producing a lightcurve mathematical function
data = snLSST.SNCosmoLC() #sncosmo.Model(source='hsiao')
#model.set(z=0.5, t0=55000., amplitude=1.e-10)

/Users/karaponder/.local/lib/python2.7/site-packages/LSSTmetrics/metrics.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lc['modelFlux'] = lc['flux']
/Users/karaponder/.local/lib/python2.7/site-packages/LSSTmetrics/metrics.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lc['deviation'] = np.random.normal(size=len(lc['flux']))


20


In [12]:
## For plotting tests
# function that takes evaluates lightcurve function given a sampling strategy
#times = [54990., 55000., 55020.]
#samples = -1.*np.array(model.bandmag('desr', 'ab', times))
#plt.plot(times,samples)

In [15]:
# function that shifts lightcurve sampling, creating many samplings of same underlying lightcurve
#for x in times:
#    model.set(z=0.5, t0=x, amplitude=1.e-10)
#    sample = -1.*np.array(model.bandmag('desr', 'ab', times))
#    plt.plot(times,sample)
#plt.show()

In [15]:
# function that fits lightcurves to samples, producing some set of parameters and variances
# first try maximum likelihood method of sncosmo.fit_lc
#data = sncosmo.load_example_data()

model = sncosmo.Model(source='salt2-extended')

#mcmc_out = sncosmo.fit_lc(data,model,['z', 't0', 'x0', 'x1', 'c'],bounds={'z':(0.3, 0.7)})
#print(res.errors)
# next try sampling method of sncosmo.mcmc_lc
mcmc_out = sncosmo.mcmc_lc(data,model,['z', 't0', 'x0', 'x1', 'c'],bounds={'z':(0.3, 0.7)})
#print(res.errors)

In [17]:
#t = anf.ResChar.fromSNCosmoRes(mcmc_out)
#samples = t.salt_samples()
#mu = samples['mu']

(       errors: OrderedDict([('z', 0.02978110321019567), ('t0', 0.6706340320961317), ('x0', 2.8197214166691003e-07), ('x1', 0.3511095350335572), ('c', 0.05453045604157661)])
   parameters: array([  5.04327426e-01,   4.95700436e+04,   8.94929586e-06,
         1.32433316e-02,   2.14689961e-02])
      success: True
         ndof: 15
   covariance: array([[  8.93551032e-04,   1.37791936e-02,  -7.22520818e-10,
          1.73195160e-03,  -1.25847576e-03],
       [  1.37791936e-02,   4.49759590e-01,  -5.11834555e-08,
          1.00655758e-01,  -2.23722465e-02],
       [ -7.22520818e-10,  -5.11834555e-08,   7.95082887e-14,
         -5.78691000e-08,   7.82384516e-10],
       [  1.73195160e-03,   1.00655758e-01,  -5.78691000e-08,
          1.23277906e-01,   3.05761805e-04],
       [ -1.25847576e-03,  -2.23722465e-02,   7.82384516e-10,
          3.05761805e-04,   2.97357064e-03]])
 vparam_names: ['z', 't0', 'x0', 'x1', 'c']
        chisq: 0.0256900174467948
  param_names: ['z', 't0', 'x0', 'x1', 

In [19]:
#var_mu = t.salt_samples().mu.std()*t.salt_samples().mu.std()

In [ ]:
# function that calculates bias of fit parameters relative to true parameters

In [63]:
print bumps

[]
